<a href="https://colab.research.google.com/github/viabloko/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,063 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,398 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,572 kB in 1s (1,793 kB/s)
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_4_bed = spark.sql("""Select round(avg(price), 2), year(date) from home_sales where bedrooms == 4 group by year(date)""")
avg_4_bed.show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|            300263.7|      2019|
|           298353.78|      2020|
|           301819.44|      2021|
+--------------------+----------+



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_3_bed_3_bath = spark.sql("""Select round(avg(price), 2), year(date) from home_sales where bedrooms == 3 and bathrooms == 3 group by year(date)""")
avg_3_bed_3_bath.show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           292725.69|      2022|
|           287287.82|      2019|
|           294204.16|      2020|
|           294211.46|      2021|
+--------------------+----------+



In [14]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_3_bed_3_bath_2_floor = spark.sql("""Select round(avg(price), 2), year(date) from home_sales where bedrooms == 3 and bathrooms == 3 and floors == 2 and sqft_living > 2000 group by year(date)""")
avg_3_bed_3_bath_2_floor.show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           289517.82|      2022|
|           289976.57|      2019|
|           292289.09|      2020|
|           296330.96|      2021|
+--------------------+----------+



In [17]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""Select view, round(avg(price), 2) from home_sales group by view having round(avg(price), 2) >= 350000""")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.08683490753173828 seconds ---


In [18]:
# 7. Cache the the temporary table
spark.sql("cache table home_sales")

DataFrame[]

In [19]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [20]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""Select view, round(avg(price), 2) from home_sales group by view having round(avg(price), 2) >= 350000""")

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.04424476623535156 seconds ---


In [21]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('parquet_home_sales', mode='overwrite')

In [22]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_home_sales')

In [23]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [24]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""Select view, round(avg(price), 2) from p_home_sales group by view having round(avg(price), 2) >= 350000""")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.028961896896362305 seconds ---


In [25]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [26]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False